In [3]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
x = []
y = []
z = []
residue = []
NO_aminoacid = []
for line in open('1uud.pdb'):
    list = line.split()
    if list[0] == 'ATOM' or 'HETATM':
        residue.append(list[3])
        type_of_chain = list[4]
        NO_aminoacid.append(list[5])
        x.append(float(list[6]))
        y.append(float(list[7]))
        z.append(float(list[8]))
number_of_atom = len(x)
#得到序列位点
myseq = 'GGCAGAUCUGAGCCUGGGAGCUCUCUGCC'
def Kmers_funct(seq, size): 
    return [seq[x:x+size] for x in range(len(seq) - size + 1)]
zimu = Kmers_funct(myseq, 1)

#计算子代的degree
number_of_aminoacid = 1
Rev_NO_aminoacid = [1]*number_of_atom
for i in range(2,len(x)):
    if NO_aminoacid[i]!=NO_aminoacid[i-1]:
        number_of_aminoacid = number_of_aminoacid +1
        for j in range(i,len(x)):
            Rev_NO_aminoacid[j] = number_of_aminoacid
            
contact = np.zeros((number_of_aminoacid, number_of_aminoacid)).astype('int64')
for i in range(len(x)):
    for j in range(len(x)):
        if abs(Rev_NO_aminoacid[i]-Rev_NO_aminoacid[j]) > 1:
            a=[x[i],y[i],z[i]]
            b=[x[j],y[j],z[j]]
            X=np.vstack([a,b]) 
            d_ij = pdist(X)
            if d_ij <= 8 :
                contact[Rev_NO_aminoacid[i]-1][Rev_NO_aminoacid[j]-1] = 1
degree = []
for i in range(len(contact)):
    degree.append(sum(contact[i]))

#
import copy
D=copy.deepcopy(contact)
ilter=[i for i in range(len(contact))]

for o in ilter:
    for d in ilter:
        if d==o:
            continue
        if D[o][d]==0:
            D[o][d]=999
#使用Floyd算法计算SP
for mid in ilter:
    for o in ilter:
        for d in ilter:
            if D[o][mid]!=999 and D[o][mid]!=999 and D[o][d]>D[o][mid]+D[mid][d]:
                D[o][d]=D[o][mid]+D[mid][d]     

closeness = []
for i in range(len(D)):
    closeness.append((len(D)-1)/sum(D[i]))

#计算子代的位点
son_of_site = []
for i in range(len(contact)):
    site = []
    for j in range(len(contact)):
        if contact[i][j] == 1:
            site.append(j+1)
    son_of_site.append(site)
    
#计算子代的degree & closeness & ASAS
a1 = []
a2 = []
for i in range(len(son_of_site)):
    b1 = []
    b2 = []
    for j in range(len(son_of_site[i])):
        b1.append(degree[son_of_site[i][j]-1])
        b2.append(closeness[son_of_site[i][j]-1])
    a1.append(b1)
    a2.append(b2)
    
degree_of_son = []
closeness_of_son = []
for i in range(len(a1)):
    degree_of_son.append(sum(a1[i]))
    closeness_of_son.append(sum(a2[i]))
####a1列表是每个位点的各个子代位点，a2列表是每个位点与各个子代之间的closeness
BEE = {'A': [0,1,1,1],
       'U': [1,1,1,0],
       'G': [1,1,0,1],
       'C': [1,0,1,1],}
be2 = []
for i in range(len(a1)):
    be1 = []
    for j in range(len(a1[i])):
        be1.append(BEE.get(zimu[a1[i][j]-1]))
    be2.append(be1)

be5 = []
be6 = []
for i in range(len(be2)):
    be3 = []
    be4 = []
    for j in range(len(be2[i])):
        be3.append([a1[i][j] * k for k in be2[i][j]])
        be4.append([a2[i][j] * k for k in be2[i][j]])
    be5.append(be3)
    be6.append(be4)
    

be7 = []
be8 = []
for i in range(len(be5)):
    m1 = np.array(be5[i])
    n1 = np.array(be6[i])
    be7.append(m1.sum(axis=0))
    be8.append(n1.sum(axis=0))

    
be9 = []#转回list
be10 = []
be11 = []
be12 = []
for i in range(len(be7)):
    be9.append(be7[i].tolist())
    be10.append(be8[i].tolist())
    for j in range(4):
        be11.append(be9[i][j])
        be12.append(be10[i][j])

s11 = [i for i in be11]
s12 = [i for i in be12]
n = 4 
be_dg = [s11[i:i + n] for i in range(0, len(s11), n)]
be_cl = [s12[i:i + n] for i in range(0, len(s12), n)]

df_be_dg = pd.DataFrame(be_dg,columns=['be_dg_1','be_dg_2','be_dg_3','be_dg_4'])
df_be_cl = pd.DataFrame(be_cl,columns=['be_cl_1','be_cl_2','be_cl_3','be_cl_4'])


EIIP = {'A': 0.1260,
        'C': 0.1340,
        'G': 0.0806,
        'U': 0.1335}
eiip=[]
for i in range(len(zimu)):
    eiip.append(EIIP.get(zimu[i]))

    
#计算该位置核苷酸到此出现的频率（密度）
dst=[]
for i in range(len(myseq)):
    a=myseq[:i+1]#将a分成多段
    dst.append(a.count(a[i])/(i+1))###a[i]到此出现的次数

#该核苷酸在链上的总频率
fre = []
for i in range(len(zimu)):
    fre.append(zimu.count(zimu[i])/len(zimu))
    
cb = {'A': 2.0,'U': 2.0,'G': 3.0,'C': 3.0}

cb2 = []
eiip2 = []
dst2 = []
fre2 = []
for i in range(len(a1)):
    cb1 = []
    eiip1 = []
    dst1 = []
    fre1 = []
    for j in range(len(a1[i])):
        cb1.append(cb.get(zimu[a1[i][j]-1]))
        eiip1.append(eiip[a1[i][j]-1])
        dst1.append(dst[a1[i][j]-1])
        fre1.append(fre[a1[i][j]-1])
    cb2.append(cb1)
    eiip2.append(eiip1)
    dst2.append(dst1)
    fre2.append(fre1)
    
    
cb5 = []
cb6 = []
eiip5 = []
eiip6 = []
dst5 = []
dst6 = []
fre5 = []
fre6 = []
for i in range(len(cb2)):
    cb3 = []
    cb4 = []
    eiip3 = []
    eiip4 = []
    dst3 = []
    dst4 = []
    fre3 = []
    fre4 = []
    for j in range(len(cb2[i])):
        cb3.append(a1[i][j] * cb2[i][j])
        cb4.append(a2[i][j] * cb2[i][j])
        eiip3.append(a1[i][j] * eiip2[i][j])
        eiip4.append(a2[i][j] * eiip2[i][j])
        dst3.append(a1[i][j] * dst2[i][j])
        dst4.append(a2[i][j] * dst2[i][j])
        fre3.append(a1[i][j] * fre2[i][j])
        fre4.append(a2[i][j] * fre2[i][j])
    cb5.append(cb3)
    cb6.append(cb4)
    eiip5.append(eiip3)
    eiip6.append(eiip4)
    dst5.append(dst3)
    dst6.append(dst4)
    fre5.append(fre3)
    fre6.append(fre4)
    
cb7 = []
cb8 = []
eiip7 = []
eiip8 = []
dst7 = []
dst8 = []
fre7 = []
fre8 = []
for i in range(len(cb5)):
    cb7.append(sum(cb5[i])/len(cb5))
    cb8.append(sum(cb6[i]))
    eiip7.append(sum(eiip5[i])/len(eiip5))
    eiip8.append(sum(eiip6[i]))
    dst7.append(sum(dst5[i])/len(dst5))
    dst8.append(sum(dst6[i]))
    fre7.append(sum(fre5[i])/len(fre5))
    fre8.append(sum(fre6[i]))


zongjie = {'DG_son':degree_of_son,
           'CL_son':closeness_of_son,
           'cb_dg':cb7,
           'cb_cl':cb8,
           'eiip_dg':eiip7,
           'eiip_cl':eiip8,
           'dst_dg':dst7,
           'dst_cl':dst8,
           'fre_dg':fre7,
           'fre_cl':fre8,
           'be_dg_1':df_be_dg.iloc[:,0],
           'be_dg_2':df_be_dg.iloc[:,1],
           'be_dg_3':df_be_dg.iloc[:,2],
           'be_dg_4':df_be_dg.iloc[:,3],
           'be_cl_1':df_be_cl.iloc[:,0],
           'be_cl_2':df_be_cl.iloc[:,1],
           'be_cl_3':df_be_cl.iloc[:,2],
           'be_cl_4':df_be_cl.iloc[:,3]}
df = pd.DataFrame(zongjie)
df.to_csv("CN.csv", index=False ,header = None)#